In [ ]:
!pip install huggingface_hub
!pip install langchain_community
!pip install langchain


In [6]:
#insecure plugin vulnerability that reveals chat history due to exec() function
#custom_action:print(user_db["hus"]["context"])

import json
from langchain import HuggingFaceHub
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

# Initialize HuggingFaceHub
hf_access_token = os.getenv("HF_ACCESS_TOKEN")
if not hf_access_token:
    raise ValueError("HF_ACCESS_TOKEN environment variable not set or empty.")


llm = HuggingFaceHub(
    repo_id="google/flan-t5-large", 
    task="text2text-generation",  # Explicitly specify the task
    huggingfacehub_api_token=hf_access_token
)
#"mistralai/Mistral-7B-v0.1"
# File where user contexts (chat history) are stored
USER_CONTEXT_FILE = 'user_context.json'

# Load user context (chat history) from a JSON file
def load_user_context():
    if os.path.exists(USER_CONTEXT_FILE):
        with open(USER_CONTEXT_FILE, 'r') as f:
            return json.load(f)
    return {}  # Return an empty dictionary if file does not exist

# Save user context (chat history) to a JSON file
def save_user_context(user_db):
    with open(USER_CONTEXT_FILE, 'w') as f:
        json.dump(user_db, f)

# Initialize user context database from persistent storage
user_db = load_user_context()

# Global variable to track the currently logged-in user
current_user = None

# User login system
def login():
    global current_user
    username = input("Enter username: ")
    password = input("Enter password: ")
    
    # Check if user exists and password matches
    if username in user_db and user_db[username]["password"] == password:
        print(f"Welcome back, {username}! Resuming your previous session.")
        current_user = username
        return True
    else:
        print("Invalid credentials!")
        return False

# Utility function: dynamically execute custom actions to manage context
def dynamic_code_execution(custom_code):
    try:
        exec(custom_code)
    except Exception as e:
        return f"Error: {str(e)}"
    return "Action completed."

# Core function to process user input and generate chatbot responses
def ask_model(user_input):
    global user_db, current_user

    # Save user input as part of the user's ongoing chatbot context
    user_db[current_user]["context"].append(f"User: {user_input}")
    
    # Save updated context to file
    save_user_context(user_db)

    # Custom actions for context management (for developers' use)
    if user_input.startswith("custom_action:"):
        code = user_input.split("custom_action:")[1].strip()
        return dynamic_code_execution(code)
    
    # Get the chatbot response from the LLM and update context
    response = llm.invoke(user_input)
    user_db[current_user]["context"].append(f"Bot: {response}")
    
    # Save updated context to file
    save_user_context(user_db)

    return response

# Main loop for continuous interaction after login
def chat_with_model():
    global current_user
    print("Chatbot is ready to assist! Type 'exit' to end the session.")
    
    while True:
        user_input = input(f"{current_user}: ")
        if user_input.lower() == 'exit':
            print("Session ended. Goodbye!")
            break
        
        # Process the user input through the ask_model function
        response = ask_model(user_input)
        print(f"Bot: {response}")

# Add new users to the system
def create_user():
    global user_db
    username = input("Enter new username: ")
    password = input("Enter new password: ")
    
    if username in user_db:
        print("User already exists!")
    else:
        user_db[username] = {"password": password, "context": []}
        save_user_context(user_db)
        print(f"User {username} created successfully.")

# Login prompt and chat initialization
if __name__ == "__main__":
    print("Welcome to the Chatbot!")
    # Load existing user contexts from the file
    user_db = load_user_context()

    # Option to create a new user or login
    while True:
        choice = input("Type 'login' to sign in or 'create' to create a new user: ").lower()
        if choice == 'create':
            create_user()
        elif choice == 'login' and login():
            break
        else:
            print("Invalid choice! Try again.")
    
    # Start chatting after login
    chat_with_model()

['User: what is the color of a watermelon?', 'User: what is the color of watermelon', 'User: what is the color of watermelon', 'User: what is the color of watermelon?', 'User: hello', 'User: orange?', 'Bot: oranges are a fruit', 'User: what is the color of the watermelon?', 'Bot: greenish-yellow', 'User: what is orange', 'User: what is cola', 'User: what is orange', 'Bot: oranges are oranges']
Bot: Action completed.
Session ended. Goodbye!


In [ ]:
#Solution for Insecure Plugin Vulnerability
#custom_action:fetch_user_context hus
from langchain import HuggingFaceHub
import os
from dotenv import load_dotenv
import re

# Load environment variables
load_dotenv(override=True)

# Initialize HuggingFaceHub
hf_access_token = os.getenv("HF_ACCESS_TOKEN")
if not hf_access_token:
    raise ValueError("HF_ACCESS_TOKEN environment variable not set or empty.")

llm = HuggingFaceHub(
    repo_id="google/flan-t5-large", 
    task="text2text-generation",  # Explicitly specify the task
    huggingfacehub_api_token=hf_access_token
)

# File where user contexts (chat history) are stored
USER_CONTEXT_FILE = 'user_context.json'

# Load user context (chat history) from a JSON file
def load_user_context():
    if os.path.exists(USER_CONTEXT_FILE):
        with open(USER_CONTEXT_FILE, 'r') as f:
            return json.load(f)
    return {}  # Return an empty dictionary if file does not exist

# Save user context (chat history) to a JSON file
def save_user_context(user_db):
    with open(USER_CONTEXT_FILE, 'w') as f:
        json.dump(user_db, f)

# Initialize user context database from persistent storage
user_db = load_user_context()

# Global variable to track the currently logged-in user
current_user = None

# Allowed actions for dynamic execution
allowed_actions = {
    "manage_context": lambda: "Managing context safely.",
    "fetch_user_context": lambda: "Fetching user context securely."
}

# User login system
def login():
    global current_user
    username = input("Enter username: ")
    password = input("Enter password: ")
    
    # Check if user exists and password matches
    if username in user_db and user_db[username]["password"] == password:
        print(f"Welcome back, {username}! Resuming your previous session.")
        current_user = username
        return True
    else:
        print("Invalid credentials!")
        return False

# Utility function: dynamically execute custom actions from a whitelist
def dynamic_code_execution(custom_code):
    if custom_code in allowed_actions:
        try:
            result = allowed_actions[custom_code]()
            return f"Action completed: {result}"
        except Exception as e:
            return f"Error: {str(e)}"
    else:
        return "Error: Invalid action!"

# Input validation to ensure no malicious code
def sanitize_input(user_input):
    # Allow letters, numbers, underscores, colons, and spaces
    if re.match("^[a-zA-Z0-9_: ]*$", user_input):
        return user_input
    else:
        raise ValueError("Invalid input. Only letters, numbers, colons, and underscores are allowed.")

# Role-Based Access Control (RBAC)
def check_user_access(user_target):
    global current_user
    if current_user != user_target:
        raise PermissionError("You are not authorized to access this data.")

# Fetch user context securely (RBAC protected)
def fetch_user_context(target_user):
    try:
        # Check if the logged-in user is allowed to access the target user's context
        check_user_access(target_user)
        return f"Context for {target_user}: {user_db[target_user]['context']}"
    except PermissionError as e:
        return str(e)

# Core function to process user input and generate chatbot responses
def ask_model(user_input):
    global user_db, current_user

    # Validate and sanitize user input
    try:
        sanitized_input = sanitize_input(user_input)
    except ValueError as e:
        return str(e)

    # Save user input as part of the user's ongoing chatbot context
    user_db[current_user]["context"].append(f"User: {sanitized_input}")
    
    # Save updated context to file
    save_user_context(user_db)

    # Custom actions for context management (for developers' use)
    if sanitized_input.startswith("custom_action:"):
        code = sanitized_input.split("custom_action:")[1].strip()
        
        # Check if fetching another user's context
        if code.startswith("fetch_user_context"):
            target_user = code.split()[-1]  # Assuming the target username is passed as an argument
            return fetch_user_context(target_user)
        else:
            return dynamic_code_execution(code)
    
    # Get the chatbot response from the LLM and update context
    response = llm.invoke(sanitized_input)
    user_db[current_user]["context"].append(f"Bot: {response}")
    
    # Save updated context to file
    save_user_context(user_db)

    return response

# Main loop for continuous interaction after login
def chat_with_model():
    global current_user
    print("Chatbot is ready to assist! Type 'exit' to end the session.")
    
    while True:
        user_input = input(f"{current_user}: ")
        if user_input.lower() == 'exit':
            print("Session ended. Goodbye!")
            break
        
        # Process the user input through the ask_model function
        response = ask_model(user_input)
        print(f"Bot: {response}")

# Add new users to the system
def create_user():
    global user_db
    username = input("Enter new username: ")
    password = input("Enter new password: ")
    
    if username in user_db:
        print("User already exists!")
    else:
        user_db[username] = {"password": password, "context": []}
        save_user_context(user_db)
        print(f"User {username} created successfully.")

# Login prompt and chat initialization
if __name__ == "__main__":
    print("Welcome to the Chatbot!")
    # Load existing user contexts from the file
    user_db = load_user_context()

    # Option to create a new user or login
    while True:
        choice = input("Type 'login' to sign in or 'create' to create a new user: ").lower()
        if choice == 'create':
            create_user()
        elif choice == 'login' and login():
            break
        else:
            print("Invalid choice! Try again.")
    
    # Start chatting after login
    chat_with_model()
